In [4]:
IMG = 125

In [5]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 

import torch_geometric as thg 
import torch_geometric.nn as gnn  
from torch_geometric.data import Data 
from torch_geometric.loader import DataLoader 

import h5py
import cv2
import numpy as np
import ipyvolume as ipv
from sklearn.cluster import DBSCAN
import networkx as nx
import matplotlib.pyplot as plt

PATH = r"/Users/suyashsachdeva/Desktop/gsoc_data.hdf5"

In [6]:
with h5py.File(PATH, 'r') as f:
    images = f['X_jets'][:]
    m0s = f["m0"][:]
    pts = f["pt"][:]
    ydata   = f["y"][:]

In [8]:
dataset_points = []
for image, m0, pt in  zip(images, m0s, pts):
    points_graph = []
    for c, color in enumerate(["red", "green", "blue"]):
        tracks_img = image[:, :, c] 

        y_coords, x_coords = np.where(tracks_img > 0)  
        try:
            magnitudes = tracks_img[y_coords, x_coords] 
            z_coords = np.array((m0 + pt) * (magnitudes / np.max(magnitudes)), dtype="int8")*4 
        except:
            continue

        
        if color=="red":
            for c in range(len(x_coords)):
                points_graph.append([x_coords[c], y_coords[c], z_coords[c], 1, color])
        # Form the 3D points array
        else:
            points = np.vstack((x_coords, y_coords, z_coords)).T
        
            clustering = DBSCAN(eps=1.0, min_samples=4).fit(points)
            labels = clustering.labels_

            # Filter out noise (-1 label)
            points_filtered = points[labels != -1]
            labels_filtered = labels[labels != -1]

            # Calculate centroid and radius for each cluster
            unique_labels = set(labels_filtered)
            centroids = np.array([points_filtered[labels_filtered == label].mean(axis=0) for label in unique_labels])
            radii = np.array([np.sqrt((points_filtered[labels_filtered == label].shape[0]) / np.pi) for label in unique_labels])

            # Normalize radii for visualization purposes
            try:
                radii_normalized = radii / np.max(radii) * 2  # Scale radii for better visualization
                for centroid, radius in zip(centroids, radii_normalized):
                    points_graph.append([x_coords[c], y_coords[c], z_coords[c], radius, color])
            except:
                pass
    dataset_points.append(points_graph)

In [9]:
graph_data = []
for points_graph in dataset_points:
    G = nx.Graph()
    for i, point in enumerate(points_graph):
        features = np.zeros(4, dtype="float32")
        if point[-1]=="red":
            features[0] = 1.0
        elif point[-1]=="blue":
            features[1] = 1.0 
        else: 
            features[2] = 1.0
        features[-1] = point[3]-1.0
        G.add_node(i, x=features)

    # Define a threshold distance for connecting nodes
    threshold_distance = 60.0

    # Add edges based on distance
    for i in range(len(points_graph)):
        for j in range(i+1, len(points_graph)):
            dist = np.linalg.norm(np.array(points_graph[i][:3]) - np.array(points_graph[j][:3]))
            if dist <= threshold_distance:
                # The connection strength could be inversely proportional to the distance
                strength = 1 / dist if dist != 0 else 1
                G.add_edge(i, j, weight=strength)
    graph_data.append(G)

In [10]:
G.nodes[0]["x"]

array([1., 0., 0., 0.], dtype=float32)

In [11]:
dataset = []
for G, Y in zip(graph_data, ydata):    
    node_features = torch.tensor([G.nodes[node]["x"] for node in G.nodes()], dtype=torch.float)
    edge_list = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
    edge_weights = torch.tensor([G[u][v]['weight'] for u, v in G.edges()], dtype=torch.float)
    y = torch.tensor(Y)
    data = Data(x=node_features, edge_index=edge_list, edge_attr=edge_weights, y=y)
    dataset.append(data)

<ipython-input-11-b917f280ec3e>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403216189/work/torch/csrc/utils/tensor_new.cpp:278.)
  node_features = torch.tensor([G.nodes[node]["x"] for node in G.nodes()], dtype=torch.float)


In [12]:
dataset = [data for data in DataLoader(dataset, batch_size=1, shuffle=True)]

In [24]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_dense_adj, dense_to_sparse

class GCNBlock(nn.Module):
    def __init__(self, indim, outdim, num_clusters):
        super(GCNBlock, self).__init__()
        self.gcn1 = gnn.GraphSAGE(indim, outdim,num_layers=2)
        self.norm1 = gnn.BatchNorm(outdim, momentum=0.9)
        self.gcn2 = gnn.GraphSAGE(outdim, outdim, num_layers=2)
        self.norm2 = gnn.BatchNorm(outdim, momentum=0.9)
        self.gcn3 = gnn.GraphSAGE(outdim, outdim, num_layers=2)
        self.norm3 = gnn.BatchNorm(outdim, momentum=0.9)
        self.assignment_matrix_layer = nn.Linear(outdim, num_clusters)

    def forward(self, h, g, g_w, mask=None):
        h0 = torch.relu(self.norm1(self.gcn1(h, g, g_w)))
        h = torch.relu(self.norm2(self.gcn2(h0, g, g_w)))
        h = torch.relu(self.norm3(self.gcn3(h, g, g_w))) + h0

        adj = self.adj_from_edge(g, g_w)
        
        s = self.assignment_matrix_layer(h)
        h_pool, adj_pool, _, _ = gnn.dense_mincut_pool(h, adj, s, mask=mask)
        adj_pool = adj_pool.squeeze()
        s_softmax = F.softmax(s, dim=-1)
        h_pool = torch.matmul(s_softmax.t(), h)
        g_pool, g_w_pool = self.edges_from_adj(adj_pool)

        g = torch.tensor(g, dtype=torch.long)
        return h_pool,g_pool, g_w_pool 

    def edges_from_adj(self, adj):
        edges = []
        edge_weights = []

        for i in range(adj.size(0)):
            for j in range(adj.size(1)):
                weight = adj[i, j]
                if weight != 0:
                    edges.append([i, j])
                    edge_weights.append(weight)

        edge_index = torch.tensor(edges, dtype=torch.long).t()
        edge_weight = torch.tensor(edge_weights, dtype=torch.float)

        return edge_index, edge_weight

    def adj_from_edge(self, edge_index, edge_weight):
        num_nodes = torch.max(edge_index).item() + 1
        adj = torch.zeros((num_nodes, num_nodes), dtype=torch.float32)

        for i, (src, dest) in enumerate(edge_index.t()):
            adj[src, dest] = edge_weight[i]

        return adj

class Classifier(nn.Module):
    def __init__(self, in_dim=4, hidden_dim=16, clusters=128):
        super(Classifier, self).__init__()
        self.conv1 = GCNBlock(in_dim, hidden_dim, clusters)
        self.conv2 = GCNBlock(hidden_dim, hidden_dim*2, clusters//2)
        self.conv3 = GCNBlock(hidden_dim*2, hidden_dim*4, clusters//4)
        self.conv4 = GCNBlock(hidden_dim*4, hidden_dim*8, clusters//8)
        self.conv5 = GCNBlock(hidden_dim*8, hidden_dim*16, clusters//16)
        self.dense = nn.Linear(hidden_dim*16, hidden_dim)
        self.drop = nn.Dropout(0.2)
        self.classify = nn.Linear(hidden_dim, 1)

    def forward(self, g, h, g_w, batch):
        h, g, g_w = self.conv1(h, g, g_w)
        h, g, g_w = self.conv2(h, g, g_w)
        h, g, g_w = self.conv3(h, g, g_w)
        h, g, g_w = self.conv4(h, g, g_w)
        h, g, g_w = self.conv5(h, g, g_w)
        h = self.drop(gnn.global_mean_pool(h, batch=batch))
        h = torch.relu(self.dense(h))
        return torch.sigmoid(self.classify(h))
    
model = Classifier()


In [25]:
def get_accuracy(y_true, y_prob):
    assert y_true.size() == y_prob.size()
    y_prob = y_prob > 0.5
    return (y_true == y_prob).sum().item() / y_true.size(0)

In [27]:
SPLIT = 0.7
epochs = 100
criterian = nn.BCELoss()
learning_rate = 1e-3
decay = 1e-2
for epoch in range(epochs):
    learning_rate = learning_rate/(epoch*decay+1)
    print(f"Epoch: {epoch+1}/{epochs} || Learning_rate: {learning_rate}")
    
    lss = 0
    acc = 0
    vls = 0
    vac = 0
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    trainum = len(dataset)*SPLIT
    for step, xtr in enumerate(dataset):
            # try:
                h = xtr.x
                g = xtr.edge_index
                g_w = xtr.edge_attr
                batch = xtr.batch
                ytr = xtr.y.reshape(-1,1 )

                ypred = model(g,h, g_w, batch)
                
                loss = criterian(ypred.reshape(-1, 1), ytr.reshape(-1,1))
                if step<trainum:
                    acc = acc+ get_accuracy(ypred, ytr)
                    lss = lss+loss
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                else:
                        vac = vac + get_accuracy(ypred, ytr)
                        vls = vls+loss                         
        
    print(f"Dataset:  Loss: {lss/(trainum+1):.4f} || Accuracy: {acc/(trainum+1):.4f} || Validation Loss: {vls/(step-trainum+1):.4f} || Validation Accuracy: {vac/(step-trainum+1):.4f}")
    print("\n")

Epoch: 1/100 || Learning_rate: 0.001


<ipython-input-24-dba70fee09f5>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  g = torch.tensor(g, dtype=torch.long)


RuntimeError: Expected index [76] to be smaller than self [1] apart from dimension 0 and to be smaller size than src [8]